In [ ]:
pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 45kB/s 


In [ ]:
pip install pandas-ods-reader

     |████████████████████████████████| 133kB 2.8MB/s 
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49001 sha256=7cf06e92a6f2997dbb45dacc9cbc1d2adb4cb4bd860b7b4a5e5c45b8dee00d78
  Stored in directory: /root/.cache/pip/wheels/c9/6c/f9/107d39d45441980bf273757eba003ef057c6193c9d7650fac7
Successfully built ezodf


In [ ]:
pip install unrar

In [ ]:
pip install pyunpack

In [ ]:
pip install patool

     |████████████████████████████████| 81kB 2.2MB/s 


In [ ]:
from pyunpack import Archive
Archive('mrlEyes_2018_01.rar').extractall('sample_data/')

In [ ]:
pip install keras_preprocessing

In [ ]:
import pandas as pd
import cv2
import os
import sys
from pandas_ods_reader import read_ods
import ezodf
from sklearn.model_selection import train_test_split
import numpy as np

class Dataset():

    def __init__(self):
        self.path = "sample_data/mrlEyes_2018_01/"
        self.X_train = None
        self.Y_train = None
        self.X_test = None
        self.Y_test = None
        self.IMG_SIZE = 64
        self.df = None

    def read_ods(self, header):
        doc = ezodf.opendoc(self.path+"stats_2018_01.ods").sheets[0]
        # sheet = doc.sheets
        # df_dict = {}

        return pd.DataFrame({col[header].value: [x.value for x in col[header + 1:]]
                             for col in doc.columns()})
    
    def make_dataset(self):
        temp = []
        data = []
        for root, dirs, files in os.walk(self.path):
            for file in files:
                if file is not None:
                    temp.append(file)

        for file in temp:

            if file == 'annotation.txt':
                temp.remove(file)
            if file == 'stats_2018_01.ods':
                temp.remove(file)

        for i, file_dt in enumerate(temp):
            index = i
            label = file_dt.split("_")[4]
            img_name = file_dt
            folder = file_dt.split("_")[0]
            filename = folder + "/" + img_name
            img = cv2.imread(os.path.join(self.path, filename))
            # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # new_img = cv2.resize(gray, (self.IMG_SIZE, self.IMG_SIZE))
            # cv2.imshow("Image",new_img)
            # cv2.waitKey(0)
            # print(new_img.shape)
            pair = [index, img_name, label, folder]
            data.append(pair)
        self.df = pd.DataFrame(data, columns=["Index", "Image", "Eye_state", "Folder"])
        self.df.index = self.df["Index"]
        # return self.df

    def traintest_split_data(self):
        # self.df.drop(self.df.columns[0], axis=1)
        # print(self.df)
        X = self.df["Image"]
        # Y = self.df["Eye_state"]
        folder = self.df["Folder"]
        # X_new = []

        # for i, image in enumerate(X):
        #     filename = folder[i] + "/" + image

        #     img = cv2.imread(os.path.join(self.path, filename))
        #     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #     new_img = cv2.resize(gray, (self.IMG_SIZE, self.IMG_SIZE))
        #     # cv2.imshow("Image",new_img)
        #     # cv2.waitKey(0)
        #     # print(new_img.shape)
        #     X_new.append(new_img)

        temp_train_data , test_data = train_test_split(self.df, test_size = 0.1, shuffle=False)
        train_data , val_data = train_test_split(temp_train_data, test_size = 0.1, shuffle=False)
        print(train_data)
        print(test_data)
        print(val_data)

        # train_dir = "Train"
        # test_dir = "Test"
        # os.mkdir(os.path.join(self.path, train_dir))
        # os.mkdir(os.path.join(self.path, test_dir))
        for i, im_name_tr in enumerate(train_data["Image"]):
            if i != 0:
                if train_data.loc[i]["Eye_state"] == '1':
                    img_fold = train_data.loc[i]["Folder"] + '/' + im_name_tr
                    dir = self.path + '/' + img_fold
                    imag = cv2.imread(dir)
                    # new_imag = cv2.cvtColor(imag, cv2.COLOR_BGR2GRAY)
                    new_imag1 = cv2.resize(imag, (self.IMG_SIZE, self.IMG_SIZE))
                    new_imag = cv2.cvtColor(new_imag1, cv2.COLOR_BGR2GRAY)
                    wr_imag = np.zeros((64,64,3))
                    wr_imag[:,:,0] = new_imag
                    wr_imag[:,:,1] = new_imag
                    wr_imag[:,:,2] = new_imag
                    cv2.imwrite('sample_data/mrlEyes_2018_01/Train_data1/1/'+im_name_tr,wr_imag)
                if train_data.loc[i]["Eye_state"] == '0':
                    img_fold = train_data["Folder"][i] + '/' + im_name_tr
                    dir = self.path + '/' + img_fold
                    imag = cv2.imread(dir)
                    # new_imag = cv2.cvtColor(imag, cv2.COLOR_BGR2GRAY)
                    new_imag1 = cv2.resize(imag, (self.IMG_SIZE, self.IMG_SIZE))
                    new_imag = cv2.cvtColor(new_imag1, cv2.COLOR_BGR2GRAY)
                    wr_imag = np.zeros((64,64,3))
                    wr_imag[:,:,0] = new_imag
                    wr_imag[:,:,1] = new_imag
                    wr_imag[:,:,2] = new_imag
                    cv2.imwrite('sample_data/mrlEyes_2018_01/Train_data1/0/'+im_name_tr,wr_imag)
            
        for j in range(test_data.index[0],test_data.index[-1]):
            if j != 0:
                if test_data.loc[j]["Eye_state"] == '1':
                    im_name_t = test_data.loc[j]["Image"]
                    img_fold = test_data.loc[j]["Folder"] + '/' + im_name_t
                    dir = self.path + '/' + img_fold
                    imag = cv2.imread(dir)
                    # new_imag = cv2.cvtColor(imag, cv2.COLOR_BGR2GRAY)
                    new_imag1 = cv2.resize(imag, (self.IMG_SIZE, self.IMG_SIZE))
                    new_imag = cv2.cvtColor(new_imag1, cv2.COLOR_BGR2GRAY)
                    wr_imag = np.zeros((64,64,3))
                    wr_imag[:,:,0] = new_imag
                    wr_imag[:,:,1] = new_imag
                    wr_imag[:,:,2] = new_imag
                    cv2.imwrite('sample_data/mrlEyes_2018_01/Test_data1/1/'+im_name_t,wr_imag)
                if test_data.loc[j]["Eye_state"] == '0':
                    im_name_t = test_data.loc[j]["Image"]
                    img_fold = test_data.loc[j]["Folder"] + '/' + im_name_t
                    dir = self.path + '/' + img_fold
                    imag = cv2.imread(dir)
                    # new_imag = cv2.cvtColor(imag, cv2.COLOR_BGR2GRAY)
                    new_imag1 = cv2.resize(imag, (self.IMG_SIZE, self.IMG_SIZE))
                    new_imag = cv2.cvtColor(new_imag1, cv2.COLOR_BGR2GRAY)
                    wr_imag = np.zeros((64,64,3))
                    wr_imag[:,:,0] = new_imag
                    wr_imag[:,:,1] = new_imag
                    wr_imag[:,:,2] = new_imag
                    cv2.imwrite('sample_data/mrlEyes_2018_01/Test_data1/0/'+im_name_t,wr_imag)
        
        for k in range(val_data.index[0],val_data.index[-1]):
            if k != 0:
                if val_data.loc[k]["Eye_state"] == '1':
                    im_name_v = val_data.loc[k]["Image"]
                    img_fold = val_data.loc[k]["Folder"] + '/' + im_name_v
                    dir = self.path + '/' + img_fold
                    imag = cv2.imread(dir)
                    # new_imag = cv2.cvtColor(imag, cv2.COLOR_BGR2GRAY)
                    new_imag1 = cv2.resize(imag, (self.IMG_SIZE, self.IMG_SIZE))
                    new_imag = cv2.cvtColor(new_imag1, cv2.COLOR_BGR2GRAY)
                    wr_imag = np.zeros((64,64,3))
                    wr_imag[:,:,0] = new_imag
                    wr_imag[:,:,1] = new_imag
                    wr_imag[:,:,2] = new_imag
                    cv2.imwrite('sample_data/mrlEyes_2018_01/Val_data1/1/'+im_name_v,wr_imag)
                if val_data.loc[k]["Eye_state"] == '0':
                    im_name_v = val_data.loc[k]["Image"]
                    img_fold = val_data.loc[k]["Folder"] + '/' + im_name_v
                    dir = self.path + '/' + img_fold
                    imag = cv2.imread(dir)
                    # new_imag = cv2.cvtColor(imag, cv2.COLOR_BGR2GRAY)
                    new_imag1 = cv2.resize(imag, (self.IMG_SIZE, self.IMG_SIZE))
                    new_imag = cv2.cvtColor(new_imag1, cv2.COLOR_BGR2GRAY)
                    wr_imag = np.zeros((64,64,3))
                    wr_imag[:,:,0] = new_imag
                    wr_imag[:,:,1] = new_imag
                    wr_imag[:,:,2] = new_imag
                    cv2.imwrite('sample_data/mrlEyes_2018_01/Val_data1/0/'+im_name_v,wr_imag)
        
        return train_data, val_data, test_data

if __name__ == "__main__":
    data = Dataset()
    # df = data.create_dataset(header=0)
    df = data.make_dataset()
    train_data, val_data, test_data = data.traintest_split_data()

       Index                         Image Eye_state Folder
Index                                                      
0          0  s0013_02028_0_1_0_0_0_01.png         0  s0013
1          1  s0013_00130_0_0_0_0_0_01.png         0  s0013
2          2  s0013_01585_0_0_0_0_0_01.png         0  s0013
3          3  s0013_00201_0_0_0_0_0_01.png         0  s0013
4          4  s0013_02346_0_1_0_0_0_01.png         0  s0013
...      ...                           ...       ...    ...
68762  68762  s0029_00482_0_0_1_0_1_01.png         1  s0029
68763  68763  s0029_00851_0_0_1_0_1_01.png         1  s0029
68764  68764  s0029_00053_0_0_0_0_0_01.png         0  s0029
68765  68765  s0029_01078_0_0_1_0_1_01.png         1  s0029
68766  68766  s0029_00275_0_0_1_0_1_01.png         1  s0029

[68767 rows x 4 columns]
       Index                         Image Eye_state Folder
Index                                                      
76408  76408  s0037_03460_1_1_0_2_0_01.png         0  s0037
76409  76409  

In [7]:
import tensorflow as tf
import keras
from keras.layers import Dense, BatchNormalization, Dropout, Activation, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.losses import categorical_crossentropy, sparse_categorical_crossentropy, binary_crossentropy
from keras.models import save_model
import pandas as pd
import numpy as np
import cv2
# from Dataset import Dataset
from keras import callbacks
from keras.callbacks import ReduceLROnPlateau
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.models import Model, load_model

class Model:

    def __init__(self, batch_size, epochs):
        self.model = None
        self.batch_size = batch_size
        self.epochs = epochs
        self.inputshape = (64,64,3)
        self.STEP_SIZE_TRAIN = None
        self.STEP_SIZE_VALID = None
        self.STEP_SIZE_TEST = None

    def build_model(self):
        self.model = Sequential()

        self.model.add(Conv2D(32, (3,3), strides=2, input_shape=self.inputshape))
        self.model.add(MaxPooling2D(pool_size=(2,2)))
        self.model.add(Activation('relu'))

        self.model.add(Conv2D(64, (3, 3), strides=2))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Activation('relu'))

        self.model.add(Conv2D(256, (3, 3), strides=2))
        self.model.add(MaxPooling2D(pool_size=(1, 1)))
        self.model.add(Activation('relu'))

        self.model.add(Flatten())

        self.model.add(Dense(512, activation='relu'))
        self.model.add(Dense(256, activation='relu'))

        self.model.add(Dense(1, activation='sigmoid'))

        # return self.model

    def build_on_resnet(self):
        self.model = Sequential()
        
        self.base_model = ResNet50(input_shape=self.inputshape, include_top=False, weights="imagenet")

        for l in self.base_model.layers:
            l.trainable = False

        self.model.add(self.base_model)

        # conv1 = Conv2D(256, (1, 1), strides=2)(self.base_model.output)
        # pool1 = MaxPooling2D(pool_size=(1, 1))(conv1)
        # act1 = Activation('relu')(pool1)
        
        self.model.add(Flatten())

        self.model.add(Dense(512, activation='relu'))
        self.model.add(Dense(256, activation='relu'))

        self.model.add(Dense(1, activation='sigmoid'))

        # self.model = Model(inputs=self.base_model, outputs=dense3)
    
    def train_model(self, X_train, Y_train):

        self.model.summary()
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # callback_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
        #                       patience=2, min_lr=0.0000001,cooldown=1)
        # checkpoint_best = callbacks.ModelCheckpoint(filepath='sample_data/best_model.h5', monitor="val_accuracy", save_best_only=True)

        self.model.fit(X_train, Y_train, batch_size=self.batch_size, epochs=self.epochs)

        print("Model Trained")

    def train_model_fit(self, train_data, val_data, callback):
        self.model.summary()
        self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

        self.model.fit(train_data, steps_per_epoch=self.STEP_SIZE_TRAIN, epochs=self.epochs, 
                       validation_data=val_data, validation_steps=self.STEP_SIZE_VALID, callbacks=callback)
    
    def train_model_gen(self, train_gen, val_gen):

        self.model.summary()
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        self.model.fit_generator(generator=train_gen, steps_per_epoch=self.STEP_SIZE_TRAIN,
                    validation_data=val_gen,
                    validation_steps=self.STEP_SIZE_VALID,
                    epochs=self.epochs)

    def evaluate_model(self, X_test, Y_test):

        loss, accuracy = self.model.evaluate(X_test, Y_test, batch_size=self.batch_size)
        print("Loss: "+str(loss))
        print("Accuracy: "+str(accuracy))

    def evaluate_model_gen(self, test_gen):

        self.model.evaluate_generator(generator=test_gen,steps=self.STEP_SIZE_VALID)

    def evaluate_new(self, test_gen):

        self.model.evaluate(generator=test_gen,steps=self.STEP_SIZE_VALID)

    def save(self):

        self.model.save('sample_data/mrlEyes_2018_01/model.h5')
        print("Model Saved")

    def predict(self, image):

        image = image.reshape((1,1, 80, 80))
        image.astype('Float32')
        image /= 255.0

        result = self.model.predict_proba(image)
        max_index = np.argmax(result)

        return max_index, result[0](max_index)

if __name__ == '__main__':
    print('TensorFlow version:', tf.__version__)
    print('Is using GPU?', tf.test.is_gpu_available())

    from tensorflow.python.client import device_lib

    # def get_available_devices():
    #     local_device_protos = device_lib.list_local_devices()
    #     return [x.name for x in local_device_protos]

    # print(get_available_devices())

    # with tf.device('/device:XLA_GPU:0'):

    model = Model(epochs=20, batch_size=32)
    # data = Dataset()
    # data.make_dataset()
    # train_data, val_data, test_data = data.traintest_split_data()
    # model.inputshape = (64, 64, 1)
    
    data_gen = ImageDataGenerator(rescale=1./255.)
    train_gen = data_gen.flow_from_directory(directory=r"sample_data/mrlEyes_2018_01/Train_data1/",
                                              target_size=(64, 64),
                                              color_mode="rgb",
                                              batch_size=32,
                                              classes = ['1', '0'],
                                              class_mode="binary",
                                              shuffle=True)

    val_gen = data_gen.flow_from_directory(directory=r"sample_data/mrlEyes_2018_01/Val_data1/",
                                              target_size=(64, 64),
                                              color_mode="rgb",
                                              batch_size=32,
                                              classes = ['1', '0'],
                                              class_mode="binary",
                                              shuffle=True)
    
    test_gen = data_gen.flow_from_directory(directory=r"sample_data/mrlEyes_2018_01/Test_data1/",
                                              target_size=(64, 64),
                                              color_mode="rgb",
                                              batch_size=32,
                                              classes = ['1', '0'],
                                              class_mode=None,
                                              shuffle=False)

    STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
    STEP_SIZE_VALID=val_gen.n//val_gen.batch_size
    STEP_SIZE_TEST=test_gen.n//test_gen.batch_size

    model.STEP_SIZE_TRAIN = STEP_SIZE_TRAIN
    model.STEP_SIZE_VALID = STEP_SIZE_VALID
    model.STEP_SIZE_TEST = STEP_SIZE_TEST

    callback_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0000001,cooldown=1)
    checkpoint_best = callbacks.ModelCheckpoint(filepath='sample_data/mrlEyes_2018_01/best_model.h5', monitor="val_accuracy", save_best_only=True)
    callbacks = [callback_lr, checkpoint_best]

    # model.build_model()
    # model.train_model_gen(train_gen, val_gen)
    model.build_on_resnet()
    model.train_model_fit(train_gen, val_gen, callbacks)
    # model.evaluate_model_gen(test_gen)
    # model.evaluate_new(test_gen)
    # model.save()

    # model = load_model("best_model.h5")
    # model.predict()
    # model.evaluate_new(val_gen)
    # model.save()

2148/2148 [==============================] - 1113s 518ms/step - loss: 0.4818 - accuracy: 0.7695 - val_loss: 0.4490 - val_accuracy: 0.8048
Epoch 2/20
Epoch 2/20
2148/2148 [==============================] - 1107s 515ms/step - loss: 0.3592 - accuracy: 0.8493 - val_loss: 0.3941 - val_accuracy: 0.8243
Epoch 3/20
Epoch 3/20
2148/2148 [==============================] - 1104s 514ms/step - loss: 0.3196 - accuracy: 0.8694 - val_loss: 0.5056 - val_accuracy: 0.7483
Epoch 4/20
Epoch 4/20
2148/2148 [==============================] - 1105s 515ms/step - loss: 0.2952 - accuracy: 0.8809 - val_loss: 0.3818 - val_accuracy: 0.8342
Epoch 5/20
Epoch 5/20
2148/2148 [==============================] - 1103s 514ms/step - loss: 0.2801 - accuracy: 0.8871 - val_loss: 0.4973 - val_accuracy: 0.7782
Epoch 6/20
Epoch 6/20
2148/2148 [==============================] - 1104s 514ms/step - loss: 0.2655 - accuracy: 0.8942 - val_loss: 0.3433 - val_accuracy: 0.8557
Epoch 7/20
Epoch 7/20
2148/2148 [=============================

TypeError: ignored

TypeError: ignored

In [11]:
model = load_model('sample_data/mrlEyes_2018_01/best_model.h5')
model.evaluate(x=val_gen, steps=STEP_SIZE_VALID)

238/238 [==============================] - 108s 455ms/step - loss: 0.3383 - accuracy: 0.8627


[0.338301420211792, 0.8626575469970703]

In [12]:
preds = model.predict(x=test_gen, verbose=1, steps=STEP_SIZE_TEST)
actual = test_gen.classes
file = open("results_new.txt", 'w+')
for i in range(len(preds)):
    val = (preds[i][0], actual[i])
    file.write(str(val))
file.close()

265/265 [==============================] - 121s 458ms/step


In [14]:
pip install matplotlib

In [15]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,40))

loses = pd.DataFrame(model.history.history)
# loses.plot()
loses.info()
# vloses = pd.DataFrame(model.history.history['val_loss'])
# vloses.plot()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

<Figure size 2880x2880 with 0 Axes>

In [16]:
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

KeyError: ignored

In [ ]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()